In [1]:
!pip install opendatasets torch torchvision torchaudio ultralytics opencv-python pandas matplotlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 57.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
import opendatasets as od

# Download dataset
od.download("https://www.kaggle.com/datasets/msambare/fer2013")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: yousefahmedsaleh
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013


100%|██████████| 60.3M/60.3M [00:03<00:00, 17.1MB/s]


In [3]:
!pip install ultralytics

In [4]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11n.pt")  # Ensure this file is in the correct directory
print("YOLO11 is ready!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 367MB/s]

YOLO11 is ready!


In [5]:
import os
import cv2
import glob
from tqdm import tqdm

# Define dataset paths
DATASET_PATH = "/content/fer2013"
OUTPUT_PATH = "/content/yolo_fer"

os.makedirs(OUTPUT_PATH, exist_ok=True)

# Define class labels
emotion_classes = {
    "angry": 0, "disgust": 1, "fear": 2, "happy": 3,
    "neutral": 4, "sad": 5, "surprise": 6
}

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def convert_to_yolo(image_path, label, image_output_dir, label_output_dir):
    if not os.path.exists(image_path):
        return False

    img = cv2.imread(image_path)
    if img is None:
        return False  # Skip if image cannot be loaded

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h, w = img.shape[:2]

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    if len(faces) == 0:
        return False  # Skip images with no detected faces

    # Save image and label
    new_img_path = os.path.join(image_output_dir, os.path.basename(image_path))
    cv2.imwrite(new_img_path, img)

    label_txt_path = os.path.join(label_output_dir, os.path.basename(image_path).replace(".jpg", ".txt"))
    with open(label_txt_path, "w") as f:
        for (x, y, w, h) in faces:
            x_center = (x + w / 2) / img.shape[1]
            y_center = (y + h / 2) / img.shape[0]
            w_norm = w / img.shape[1]
            h_norm = h / img.shape[0]
            f.write(f"{label} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

    return True

# Process images
for split in ["train", "test"]:
    input_dir = os.path.join(DATASET_PATH, split)
    image_output_dir = os.path.join(OUTPUT_PATH, "images", split)
    label_output_dir = os.path.join(OUTPUT_PATH, "labels", split)

    os.makedirs(image_output_dir, exist_ok=True)
    os.makedirs(label_output_dir, exist_ok=True)

    for emotion, label in emotion_classes.items():
        images = glob.glob(os.path.join(input_dir, emotion, "*.jpg"))

        for img_path in tqdm(images, desc=f"Processing {split}/{emotion}"):
            convert_to_yolo(img_path, label, image_output_dir, label_output_dir)

print("✅ YOLO dataset conversion completed.")


Processing test/surprise: 100%|██████████| 831/831 [00:00<00:00, 850.81it/s]

✅ YOLO dataset conversion completed.


In [6]:
data_yaml = f"""train: {OUTPUT_PATH}/images/train
val: {OUTPUT_PATH}/images/test
nc: 7
names: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
"""

with open(os.path.join(OUTPUT_PATH, "data.yaml"), "w") as f:
    f.write(data_yaml)

print("✅ YOLO dataset is ready!")


✅ YOLO dataset is ready!


In [7]:
import os
import shutil

# Ensure output dataset path exists
if not os.path.exists(OUTPUT_PATH):
    print(f"❌ Error: Dataset path '{OUTPUT_PATH}' does not exist.")
    exit()

for split in ["train", "test"]:
    split_path = os.path.join(OUTPUT_PATH, split)

    # Check if the split folder exists before listing contents
    if not os.path.exists(split_path):
        print(f"⚠️ Warning: {split_path} does not exist. Skipping...")
        continue

    images_path = os.path.join(OUTPUT_PATH, "images", split)
    labels_path = os.path.join(OUTPUT_PATH, "labels", split)

    os.makedirs(images_path, exist_ok=True)
    os.makedirs(labels_path, exist_ok=True)

    # List emotion folders inside the split directory
    emotion_folders = os.listdir(split_path)

    for emotion in emotion_folders:
        emotion_folder_path = os.path.join(split_path, emotion)

        if os.path.isdir(emotion_folder_path):
            for img_file in os.listdir(emotion_folder_path):
                src = os.path.join(emotion_folder_path, img_file)
                dst = os.path.join(images_path, img_file)
                shutil.move(src, dst)

print("✅ Dataset structure fixed!")


⚠️ Warning: /content/yolo_fer/train does not exist. Skipping...
⚠️ Warning: /content/yolo_fer/test does not exist. Skipping...
✅ Dataset structure fixed!


In [8]:
from ultralytics import YOLO

# Load the model
model = YOLO("yolo11n.pt")  # Ensure this file is in the correct directory

# Train
model.train(
    data=f"{OUTPUT_PATH}/data.yaml",
    epochs=40,
    batch=16,
    imgsz=128,
    device="cuda"
)


Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/yolo_fer/data.yaml, epochs=40, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

100%|██████████| 755k/755k [00:00<00:00, 116MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/yolo_fer/labels/train... 8636 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8636/8636 [00:03<00:00, 2420.18it/s]


train: New cache created: /content/yolo_fer/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all       2167       2167      0.483      0.721      0.611      0.531



40 epochs completed in 0.790 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:09<00:00,  6.81it/s]


                   all       2167       2167      0.483      0.721      0.611      0.531
                 angry        282        282      0.389      0.745      0.591      0.505
               disgust         38         38      0.332      0.395       0.38      0.343
                  fear        255        255      0.315      0.592      0.437       0.38
                 happy        656        656      0.802      0.942      0.946      0.823
               neutral        449        449      0.516      0.844      0.711       0.62
                   sad        253        253      0.335      0.655      0.376      0.329
              surprise        234        234      0.694      0.872      0.837      0.718
Speed: 0.0ms preprocess, 0.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78755821f3d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [9]:
import os

# Locate the training results folder
runs_path = "runs/detect"
if os.path.exists(runs_path):
    print("✅ Training results found in:", os.listdir(runs_path))
else:
    print("❌ No training results found!")


✅ Training results found in: ['train']


In [10]:
from ultralytics import YOLO

model = YOLO("runs/detect/train/weights/best.pt")  # Load trained model
model.val()  # Evaluate model performance


Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,517 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/yolo_fer/labels/test.cache... 2167 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2167/2167 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 136/136 [00:10<00:00, 12.93it/s]


                   all       2167       2167      0.484      0.721      0.611      0.531
                 angry        282        282      0.389      0.745      0.591      0.505
               disgust         38         38      0.337      0.395       0.38      0.343
                  fear        255        255      0.315      0.592      0.438      0.381
                 happy        656        656      0.803      0.942      0.947      0.823
               neutral        449        449      0.516      0.846      0.711       0.62
                   sad        253        253      0.333      0.652      0.376      0.328
              surprise        234        234      0.694      0.872      0.837      0.719
Speed: 0.0ms preprocess, 1.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x787551d0e090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [ ]:
results = model.predict(source="/content/fer2013/train/angry/Training_10899258.jpg", conf=0.25)


image 1/1 /content/fer2013/train/angry/Training_10899258.jpg: 640x640 (no detections), 9.6ms
Speed: 2.9ms preprocess, 9.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
